<center> 
    <a href="https://github.com/nebuly-ai/nebullvm#how-nebullvm-works" target="_blank" style="text-decoration: none;"> How Nebullvm Works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#tutorials" target="_blank" style="text-decoration: none;"> Tutorials </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#benchmarks" target="_blank" style="text-decoration: none;"> Benchmarks </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#installation" target="_blank" style="text-decoration: none;"> Installation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> Get Started </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#optimization-examples" target="_blank" style="text-decoration: none;"> Optimization Examples </a>
</center>
<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Discord </a> |
    <a href="https://nebuly.ai/" target="_blank" style="text-decoration: none;"> Website </a> |
    <a href="https://www.linkedin.com/company/72460022/" target="_blank" style="text-decoration: none;"> LinkedIn </a> |
    <a href="https://twitter.com/nebuly_ai" target="_blank" style="text-decoration: none;"> Twitter </a>
</center>

# Accelerate fast.ai Resnet34 with nebullvm

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the open-source library nebullvm.

With nebullvm's latest API, you can speed up models up to 10 times without any loss of accuracy (option A), or accelerate them up to 20-30 times by setting a self-defined amount of accuracy/precision that you are willing to trade off to get even lower response time (option B). To accelerate your model, nebullvm takes advantage of various optimization techniques such as deep learning compilers (in both option A and option B), quantization, half accuracy, and so on (option B).

Let's jump to the code.

# Fine-tune a fast.ai model

For the tutorial, we will use a fast.ai notebook for beginners in which we will classify whether the input image contains a cat (`True` label) or a dog (`False` label). Note that this notebook is not intended to be an in-depth guide to fast.ai libraries but, rather, shows how to use nebullvm to speed up fast.ai algorithms at inference time.

First let's download a sample of images of dogs and cats.

In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224), num_workers=0)
dls.show_batch()

Let's now fine-tune the fast.ai model.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
valid_loss, error = learn.validate()

Now that we have fine-tuned the model, let's calculate the time required to run a prediction as an average over 100 tests.

In [ ]:
import time

In [ ]:
%%capture
times = []
for _ in range(100):
    st = time.time()
    preds = learn.predict(files[0])
    times.append((time.time()-st)*1000)
fastai_vanilla_time = sum(times)/len(times)

In [ ]:
print(f"Average prediction time: {fastai_vanilla_time} ms,\nPrediction: {preds}")

In [ ]:
#learn.save(".")

# Speed up fast.ai inference with nebullvm

In [ ]:
from nebullvm import optimize_torch_model

Nebullvm was built to be very easy to use. To optimize a model, you only need to specify the model, the batch size and input size for each input tensor, and a directory in which to save the optimized model. In the example, we chose the same directory in which this notebook runs.

With the latest API, there are two ways to use nebullvm:

- Option A: Accelerate the model up to ~10 times without losing in performances (accuracy/precision/etc.)
- Option B: Accelerate the model up to ~30 times with a pre-defined maximum loss in performances
    - B1: Performance is measured on your dataset with a performance metric that you specify.
    - B2: Performance is measured on your dataset using precision as a metric.
    - B3: Performance is measured using precision as a metric.

To learn more about how to use nebullvm, check out the <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> readme on GitHub </a>.

In this example, we provide the code to run option B.1 on the fast.ai model and you find commented out code for the other options as well. 

We set:

- Accuracy as performance metric
- Performance loss threshold to around 0.001

Now we prepare the dataset so that it can be processed by nebullvm.

In [ ]:
xs, ys = [], []
for i, (x, y) in enumerate(dls.train):
    if i >=10:
        break
    xs.append(x)
    ys.append(y)
xs = torch.cat(xs, dim=0)
ys = torch.cat(ys, dim=0)

In [ ]:
dl_nebullvm = [((x.unsqueeze(dim=0),), y.unsqueeze(0)) for x, y in zip(xs, ys)]

In [ ]:
original_model = learn.model

In [ ]:
## Option A: 2-10x acceleration, NO performance loss

# optimized_model = optimize_torch_model(
#     model=original_model,
#     batch_size=1,
#     input_sizes=[(3, 224, 224)],
#     save_dir=".",
#     #use_torch_api=True
# )

In [ ]:
## Option B.1: 2-30x acceleration, maximum performance loss below your 
## perf_loss_ths, where performance is measured on your dataset dl_nebullvm
## with your performance metric perf_metric.

optimized_model = optimize_torch_model(
    model=original_model,
    batch_size=1,
    save_dir=".",
    dataloader=dl_nebullvm,
    #use_torch_api=True,
    perf_loss_ths=0.005,
    perf_metric="accuracy",
)

In [ ]:
## Option B.2: 2-30x acceleration, maximum performance loss below your 
## perf_loss_ths, where performance is measured on your dataset dl_nebullvm
## using precision as a metric. Read more on GitHub on how to set perf_loss_ths.

# optimized_model = optimize_torch_model(
#     model=original_model,
#     batch_size=1,
#     save_dir=".",
#     dataloader=dl_nebullvm,
#     #use_torch_api=True,
#     perf_loss_ths=3,
# )

In [ ]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, core):
        super().__init__()
        self.core = optimized_model
    
    def forward(self, *args, **kwargs):
        res = self.core(*args, **kwargs)
        if isinstance(res, tuple) and len(res) == 1:
            res = res[0]
        return res
    
    def parameters(self, *args, **kwargs):
        yield torch.zeros(100)

In [ ]:
core_model = ModelWrapper(optimized_model)

In [ ]:
learn.model = core_model

In [ ]:
learn.dls.valid.bs = 1

In [ ]:
optz_valid_loss, optz_error = learn.validate(dl=learn.dls.valid)

Let's calculate the time required to run a prediction as an average over 100 tests.

In [ ]:
%%capture
times = []
for _ in range(100):
    st = time.time()
    preds = learn.predict(files[0])
    times.append((time.time()-st)*1000)
optimized_time = sum(times) / len(times)

In [ ]:
print(f"Average prediction time: {optimized_time} ms,\nAverage prediction: {preds}")

In [ ]:
print(f"Full precision error: {error}\nError after optimization: {optz_error}")

## Print the results

In [ ]:
# Enter here your username
your_username = "username"

In [ ]:
## Uncomment the following line for installing gputil if you are running on an NVIDIA GPU.
#!pip install gputil

In [ ]:
import cpuinfo
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
cpu_info = cpuinfo.get_cpu_info()['brand_raw']
gpu_info = "no"
if torch.cuda.is_available():
    import GPUtil
    gpus = GPUtil.getGPUs()
    gpu_info = list(gpus)[0].name

In [ ]:
message = f"""
Hello, I'm {your_username}!
I tested nebullvm with the following setup.

Hardware: {cpu_info} CPU and {gpu_info} GPU.
Model: {learn.arch.__name__} - fast.ai for image classification
Vanilla performance: {round(fastai_vanilla_time, 2)}ms
Optimized performance: {round(optimized_time, 2)}ms
Speedup: {round(fastai_vanilla_time/optimized_time, 1)}x
With error increase of {round((optz_error-error)/error*100, 1)}%
"""
print(message)

Great! Was it easy? How are the results? Do you have any comments?
Share your optimization results and thoughts with <a href="https://discord.gg/RbeQMu886J" target="_blank"> our community on Discord</a>, where we chat about nebullvm and AI acceleration.

Note that the acceleration of nebullvm depends very much on the hardware configuration and your AI model. Given the same input model, nebullvm can accelerate it by 10 times on some machines and perform poorly on others.

If you want to learn more about how nebullvm works, look at other tutorials and performance benchmarks, check out the links below or write to us on Discord.

<center> 
    <a href="https://github.com/nebuly-ai/nebullvm#how-nebullvm-works" target="_blank" style="text-decoration: none;"> How Nebullvm Works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#tutorials" target="_blank" style="text-decoration: none;"> Tutorials </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#benchmarks" target="_blank" style="text-decoration: none;"> Benchmarks </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#installation" target="_blank" style="text-decoration: none;"> Installation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#get-started" target="_blank" style="text-decoration: none;"> Get Started </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#optimization-examples" target="_blank" style="text-decoration: none;"> Optimization Examples </a>
</center>
<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Discord </a> |
    <a href="https://nebuly.ai/" target="_blank" style="text-decoration: none;"> Website </a> |
    <a href="https://www.linkedin.com/company/72460022/" target="_blank" style="text-decoration: none;"> LinkedIn </a> |
    <a href="https://twitter.com/nebuly_ai" target="_blank" style="text-decoration: none;"> Twitter </a>
</center>